In [1]:
config='GS36'
case='MPC001'
freq='1d'

year='2012'


In [2]:
## path for mdules

import sys
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/xscale")
import xscale

sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/common-lib/")
import GriddedData

## imports

import numpy as np
import numpy.ma as ma
import xarray as xr
import time

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.cm as mplcm
seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

### quick plot
import matplotlib.pyplot as plt

import glob
import os 

%matplotlib inline

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
if case == 'MPC001':
    if config == 'EU36':
        gridfile='/store/colombo/EU36/EU36-I/coordinates_EU36.nc'
        maskfile='/store/colombo/EU36/EU36-I/EU36.L75_mesh_mask.nc'
    if config == 'GS36':
        gridfile='/scratch/cnt0024/hmg2840/albert7a/GS36.L75/GS36.L75-I/GS36_coordinates.nc'
        maskfile='/scratch/cnt0024/hmg2840/albert7a/GS36.L75/GS36.L75-I/GS36.L75-MAA007_mesh_mask.nc'
if case == 'MPC002':
    if config == 'EU36':
        gridfile='/store/colombo/EU36/EU36-I.L150_1/coordinates_EU36.nc'
        maskfile='/store/colombo/EU36/EU36-I.L150_1/EU36.L150_1_mesh_mask.nc'
    if config == 'GS36':
        gridfile='/store/colombo/GS36/GS36-I.L150_1/GS36_coordinates.nc'
        maskfile='/store/colombo/GS36/GS36-I.L150_1/GS36.L150_1_mesh_mask.nc'
if case == 'MPC003':
    if config == 'EU36':
        gridfile='/store/colombo/EU36/EU36-I.L150_2/coordinates_EU36.nc'
        maskfile='/store/colombo/EU36/EU36-I.L150_2/EU36.L150_2_mesh_mask.nc'
    if config == 'GS36':
        gridfile='/store/colombo/GS36/GS36-I.L150_2/GS36_coordinates.nc'
        maskfile='/store/colombo/GS36/GS36-I.L150_2/GS36.L150_2_mesh_mask.nc'
 

In [4]:
grid=xr.open_dataset(gridfile)
navlat= grid['nav_lat']
navlon= grid['nav_lon']


In [5]:
def filt(w):
    win_box2D = w.window
    win_box2D.set(window='hanning', cutoff=54, dim=['x', 'y'], n=[54, 54])
    bw = win_box2D.boundary_weights(drop_dims=[])
    w_LS = win_box2D.convolve(weights=bw)
    w_SS=w-w_LS
    return w_SS


In [6]:
def hgrad(b,e1u,e2v):
    dbxu=(b.shift(x=-1) - b)/e1u
    dbyv=(b.shift(y=-1) - b)/e2v
    dbx=0.5*(dbxu.shift(x=-1)+dbxu.shift)
    dby=0.5*(dbyv.shift(y=-1)+dbyv.shift)
    hgradb=np.sqrt(dbx*dbx+dby*dby)
    return hgradb
    
    

In [8]:
    month='03'
    date_list=[]
    if month == '03':
        dayf=31
    if month == '09':
        dayf=30
    for day in np.arange(1,dayf+1):
        if day < 10:
            date='y'+str(year)+'m'+str(month)+'d0'+str(day)
        else:
            date='y'+str(year)+'m'+str(month)+'d'+str(day)
        date_list.append(date)
    datadir='/scratch/cnt0024/hmg2840/colombo/'+str(config)+'/'+str(config)+'-'+str(case)+'-S/'+str(freq)+'/'
    for date in date_list:
        fileT=datadir+'*/'+str(config)+'-'+str(case)+'_'+str(date)+'.'+str(freq)+'_gridT.nc'
        filebuoy='/scratch/cnt0024/hmg2840/albert7a/tmp/'+str(config)+'-'+str(case)+'_'+str(date)+'.'+str(freq)+'_buoyancy.nc'
        if not os.path.exists(filebuoy):
            print('Computing '+str(filebuoy))
            !ln -sf $maskfile mesh_hgr.nc
            !ln -sf $maskfile mesh_zgr.nc
            !ulimit -s unlimited; cdfbuoy -t $fileT votemper -s $fileT vosaline -l 1 -o $filebuoy
            !rm mesh*
    filesbuoy='/scratch/cnt0024/hmg2840/albert7a/tmp/'+str(config)+'-'+str(case)+'_y'+str(year)+'m'+str(month)+'d??.'+str(freq)+'_buoyancy.nc'
    dsbuoy=xr.open_mfdataset(filesbuoy,chunks={'time_counter':1,'y':700,'x':1000})
    buoy=dsbuoy.sobuoy
    buoy_SS=filt(buoy)
    dsmask=xr.open_dataset(maskfile)
    e1u=dsmask.e1u
    e2v=dsmask.e2v
        

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/backends/api.py:933: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


In [9]:
e1u.shape

(1, 950, 1468)

In [10]:
e2v.shape

(1, 950, 1468)

In [11]:
buoy_SS.shape

(31, 950, 1468)

In [13]:
dbxu=(buoy_SS.shift(x=-1) - buoy_SS)/e1u

IndexError: The indexing operation you are attempting to perform is not valid on netCDF4.Variable object. Try loading your data into memory first by calling .load().

In [15]:
truc=buoy_SS.shift(x=-1)

In [16]:
machin=truc-buoy_SS

In [30]:
machin.shape

(31, 950, 1468)

In [31]:
machin

<xarray.DataArray 'sobuoy' (time_counter: 31, y: 950, x: 1468)>
dask.array<sub, shape=(31, 950, 1468), dtype=float64, chunksize=(1, 700, 1000)>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2012-03-01T11:58:56 ... 2012-03-31T11:58:56
Dimensions without coordinates: y, x

In [17]:
fe1u=1/e1u

In [39]:
f=fe1u.squeeze()

In [40]:
bidule=f*machin

In [41]:
bidule.shape

(950, 1468, 31)

In [42]:
bidule2=machin*f

In [44]:
bidule2.shape

(31, 950, 1468)

In [48]:
def hgrad(b,e1u,e2v):
    fe1u=1/e1u
    fse1u=fe1u.squeeze()
    dbxu=(b.shift(x=-1) - b)*fse1u
    fe2v=1/e2v
    fse2v=fe2v.squeeze()
    dbyv=(b.shift(y=-1) - b)*fse2v
    dbx=0.5*(dbxu.shift(x=-1)+dbxu)
    dby=0.5*(dbyv.shift(y=-1)+dbyv)
    hgradb=np.sqrt(dbx*dbx+dby*dby)
    return hgradb
    
    

In [47]:
    b=buoy_SS
    fe1u=1/e1u
    fse1u=fe1u.squeeze()
    dbxu=(b.shift(x=-1) - b)*fse1u
    fe2v=1/e2v
    fse2v=fe2v.squeeze()
    dbyv=(b.shift(y=-1) - b)*fse2v


In [49]:
hgradb_SS=hgrad(buoy_SS,e1u,e2v)

In [50]:
hgradb_SS

<xarray.DataArray (time_counter: 31, y: 950, x: 1468)>
dask.array<sqrt, shape=(31, 950, 1468), dtype=float64, chunksize=(1, 700, 1000)>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2012-03-01T11:58:56 ... 2012-03-31T11:58:56
Dimensions without coordinates: y, x

In [ ]:
    hgradb_SS=hgrad(buoy_SS.load(),e1u,e2v)
    hphgradb=hgradb_SS
    hphgradb2 = hphgradb ** 2
    hphgradb2m = hphgradb2.mean(axis=0,keep_attrs=True)
    navlat2=np.array(navlat).squeeze()
    navlon2=np.array(navlon).squeeze()
    mgrd = GriddedData.grid2D(navlat=navlat2, navlon=navlon2)
    crs = GriddedData.grdCoarsener(mgrd,crs_factor=60)
    hphgradb2mc = crs.return_ravel(np.asarray(hphgradb2m))
    hphgradb2mcm = np.mean(hphgradb2mc,axis=-3)
    latcrs=crs.return_ravel(np.asarray(navlat2))
    loncrs=crs.return_ravel(np.asarray(navlon2))
    latcrsm=np.mean(latcrs,axis=-3)
    loncrsm=np.mean(loncrs,axis=-3)
    if month == '03':
        plot_fine_scale_hgradb(1e7*hphgradb2mcm,loncrsm, latcrsm,navlon,navlat,hphgradb2m,'March',month,config,case)
    if month == '09':
        plot_fine_scale_hgradb(1e7*hphgradb2mcm,loncrsm, latcrsm,navlon,navlat,hphgradb2m,'September',month,config,case)
